In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
df=pd.read_csv('../data/le_train.csv',index_col=0)

In [2]:
trainX=df.drop('readmitted',axis=1).copy()
trainY=df['readmitted']

In [3]:
para= {"learning_rate" : [0.01,0.1,1],
       "n_estimators": [200,500,1000,2000,3000,4000],
       "max_depth" : [3,4,5,6],
       "subsample" : [0.8,0.9,1],
       "colsample_bytree" : [0.8,0.9,1]}

In [4]:
xgboost = xgb.XGBRegressor()
n_iter_search = 2
ran_search_xgb=RandomizedSearchCV(estimator=xgboost, param_distributions=para,n_iter=n_iter_search,
                                  scoring='neg_mean_squared_error', cv=5, return_train_score=True)

In [7]:
import time

In [8]:
training_start = time.perf_counter()
xgbmodel=ran_search_xgb.fit(trainX,trainY)
training_end = time.perf_counter()

train_time = training_end-training_start

print("Time consumed for training: %4.3f seconds" % (train_time))

Time consumed for training: 5546.267 seconds


In [9]:
print("Best parameters found: ", ran_search_xgb.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(ran_search_xgb.best_score_)))

Best parameters found:  {'subsample': 1, 'n_estimators': 3000, 'max_depth': 5, 'learning_rate': 1, 'colsample_bytree': 0.9}
Lowest RMSE found:  0.4290928319887428


In [15]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score, roc_curve

In [38]:
df2=pd.read_csv('../data/le_test.csv',index_col=0)

In [42]:
testX = df2.drop('readmitted',axis=1)
testY = df2['readmitted']

In [51]:
# {'subsample': 1, 'n_estimators': 3000, 'max_depth': 5, 'learning_rate': 1, 'colsample_bytree': 0.9}
xReg = xgb.XGBRegressor(colsample_bytree=0.9, learning_rate=1, 
                             max_depth=5, n_estimators=3000,
                             objective='binary:hinge', subsample=1)


In [53]:
# make predictions for test data
training_start = time.perf_counter()
xgbR= xReg.fit(trainX,trainY)
training_end = time.perf_counter()

train_time = training_end-training_start

print("Time consumed for training: %4.3f seconds" % (train_time))
prediction_start = time.perf_counter()
preds = xgbR.predict(testX)
prediction_end = time.perf_counter()

prediction_time = prediction_end-prediction_start

print("Time consumed for prediction: %6.5f seconds" % (prediction_time))

Time consumed for training: 630.098 seconds
Time consumed for prediction: 1.57089 seconds


In [54]:
# evaluate predictions
acc = accuracy_score(testY, preds)
pre = precision_score(testY, preds)
rec = recall_score(testY, preds)
AUC = roc_auc_score(testY, preds)

                    
print("Accuracy: {:.4%}".format(acc))
print("Precision: {:.4%}".format(pre))
print("Recall: {:.4%}".format(rec))
print("AUC: {}".format(AUC))

Accuracy: 86.5139%
Precision: 24.6996%
Recall: 8.3938%
AUC: 0.5253309972620969


In [35]:
xgboost2 = xgb.XGBClassifier()
n_iter_search =2
ran_search_xgb=RandomizedSearchCV(estimator=xgboost, param_distributions=para,n_iter=n_iter_search,
                                 scoring='roc_auc', cv=5, return_train_score=True)
# model 2

In [49]:
training_start = time.perf_counter()
xgbmodel2 = xgboost2.fit(trainX,trainY)
training_end = time.perf_counter()

train_time = training_end-training_start

print("Time consumed for training: %4.3f seconds" % (train_time))

Time consumed for training: 4738.292 seconds


In [50]:
print("Best parameters found: ", ran_search_xgb.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(ran_search_xgb.best_score_)))

Best parameters found:  {'subsample': 1, 'n_estimators': 3000, 'max_depth': 5, 'learning_rate': 1, 'colsample_bytree': 0.9}
Lowest RMSE found:  0.4290928319887428


In [55]:
xClass = xgb.XGBClassifier(colsample_bytree=0.9, learning_rate=1, 
                             max_depth=5, n_estimators=3000,
                             objective='binary:hinge', subsample=1)

In [56]:
# make predictions for test data
training_start = time.perf_counter()
xgbC= xClass.fit(trainX,trainY)
training_end = time.perf_counter()

train_time = training_end-training_start

print("Time consumed for training: %4.3f seconds" % (train_time))
prediction_start = time.perf_counter()
preds = xgbC.predict(testX)
prediction_end = time.perf_counter()

prediction_time = prediction_end-prediction_start

print("Time consumed for prediction: %6.5f seconds" % (prediction_time))

Time consumed for training: 574.535 seconds
Time consumed for prediction: 1.74826 seconds


In [57]:
# evaluate predictions
acc = accuracy_score(testY, preds)
pre = precision_score(testY, preds)
rec = recall_score(testY, preds)
AUC = roc_auc_score(testY, preds)

                    
print("Accuracy: {:.4%}".format(acc))
print("Precision: {:.4%}".format(pre))
print("Recall: {:.4%}".format(rec))
print("AUC: {}".format(AUC))

Accuracy: 86.5139%
Precision: 24.6996%
Recall: 8.3938%
AUC: 0.5253309972620969


In [ ]:
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])